# Normality index

## Connect to the database

In [38]:
from databaseconnection import DatabaseConnection
gds = DatabaseConnection().get_database_connection()
gds.version()

'2.3.2'

## TO BE REMOVED DOESNT WORK: Create a normality index.

Create a normality index for each respondent in the data set. The normality index is calculated with the following formula:

$$
\frac{1}{n}\sum_{i=1}^{n}{p_i}
$$

Where $n$ is the number of questions answered by the respondent and $p_i$ is the percentage of respondents that have chosen the alternative for question $i$.

In [35]:
# TO BE REMOVED DOESNT WORK: 
gds.run_cypher("""
                MATCH (re:Respondent)-[:HAS_ANSWERED]->(a:Answer)-[:CHOSE]->(qa:QuestionAlternative)<-[:CONSISTS_OF]-(q:Question)
                WITH re, q, qa, COUNT(a) as answersForAlternative
                MATCH (q)-[:CONSISTS_OF]->(qa_total:QuestionAlternative)
                WITH re, q, qa, answersForAlternative, COUNT(qa_total) as totalAlternatives
                WITH re, q, toFloat(answersForAlternative) / toFloat(totalAlternatives) as p_i
                WITH re, COUNT(q) as totalQuestions, SUM(p_i) as sum_p_i
                SET re.normalityIndex = sum_p_i / totalQuestions
                RETURN re.id as respondentId, re.normalityIndex as normalityIndex
               """)

KeyboardInterrupt: 

## Create property `answersForAlternative` in `QuestionAlternative`



In [34]:

gds.run_cypher("""
                MATCH (a:Answer)-[chose:CHOSE]->(qa:QuestionAlternative)
                WITH qa as alternative, COUNT(chose) as answersForAlternative
                SET alternative.answersForAlternative = answersForAlternative
                RETURN alternative.name, alternative.answersForAlternative
               """)


,alternative.name,alternative.answersForAlternative
0,Årskurs 7,24
1,Tjej,29
2,Annat alternativ,1
3,Osäker på min sexuella läggning,2
4,Årskurs 8,20
...,...,...
1320,Saknas eller är inte aktuell,2
1321,Finns och är framtagen och reviderad årligen a...,37
1322,Finns och är framtagen och reviderad årligen i...,33
1323,Finns och är ’levande’ och aktuell i verksamhe...,12


## Create property `answersForQuestion` in `Question`

In [43]:
gds.run_cypher("""
               MATCH (q:Question)<-[iat:IS_ANSWER_TO]-(a:Answer)
               WITH q as question, COUNT(iat) as answersForQuestion
               SET question.answersForQuestion = answersForQuestion
               RETURN question.id, answersForQuestion
               ORDER BY answersForQuestion DESC
               """)

,question.id,answersForQuestion
0,6a0267b8-5220-4db3-98a6-a0d5bacc5190,2965
1,36393ae4-8a92-4c85-93b2-711f3759ba0b,2965
2,af247768-d27e-4e9b-94ac-a50e3a82e8c5,2965
3,b59bb433-0ca9-47f5-bf76-bfb97061612f,2965
4,4664611d-51f8-4f76-b398-3a0bfc988660,2965
...,...,...
285,fe32cb72-bec6-441b-8e21-aaf107792344,9
286,efd0ec8c-189e-4ab1-92fc-e647f0108436,9
287,4ac938fd-ec46-40ea-bef2-039cfc42156a,9
288,b9f669dd-f225-4ebb-96ab-aabf311e2b22,9


## Create property `normalityIndex` in `Respondent`

`normailtyIndex` is a property that is calculated from the sum of all the `Respondent`s `QuestionAlternative.answersForAlternative`/`Question.answersForQuestion`.

In [47]:
# Create property `normalityIndex` in `Respondent`

# `normailtyIndex` is a property that is calculated from the sum of all the `Respondent`s `QuestionAlternative.answersForAlternative`/`Question.answersForQuestion`.

gds.run_cypher("""
               MATCH (r:Respondent)-[ha:HAS_ANSWERED]->(a:Answer)-[c:CHOSE]->(qa:QuestionAlternative)<-[co:CONSISTS_OF]-(q:Question)
               WITH qa.name as alternative, qa.answersForAlternative as answersForAlternative, q.id as question, q.answersForQuestion as answersForQuestion, r.id as respondent
               
               """)

,alternative,answersForAlternative,question,answersForQuestion,respondent
0,Tafsat eller tagit på dig på ett sätt du tyckt...,143,aeed3878-7737-4d0f-856b-599ea75165ac,2836,ffed7c42-658e-44ac-8d00-e43aac67deca
1,Var jag eller min familj kommer från,54,96673245-f8d1-4604-957a-2daf3c86e9cd,2965,ffed7c42-658e-44ac-8d00-e43aac67deca
2,Vem jag blir kär i,121,96673245-f8d1-4604-957a-2daf3c86e9cd,2965,ffed7c42-658e-44ac-8d00-e43aac67deca
3,"Ja, någon gång",322,da4afddb-0c18-4fac-85b6-75e7087287da,2965,ffed7c42-658e-44ac-8d00-e43aac67deca
4,Stämmer ganska väl,1006,c2bf4c70-10f5-4a13-ae64-3207ed0d5e5a,2965,ffed7c42-658e-44ac-8d00-e43aac67deca
...,...,...,...,...,...
995,Stämmer mycket dåligt,135,f0792e8d-6076-4cd6-b9a7-8dad10bb6e7f,2965,fef76bd5-8447-4f20-aab9-453c1e284753
996,Leda eller tycka till om rastaktiviteter,743,b9daa102-6dc7-46a5-8fd7-daec08fb4400,2965,fef76bd5-8447-4f20-aab9-453c1e284753
997,Dela med mig om hur jag upplever tryggheten på...,1291,b9daa102-6dc7-46a5-8fd7-daec08fb4400,2965,fef76bd5-8447-4f20-aab9-453c1e284753
998,Vara med och ta fram klassreglerna tillsammans...,1129,b9daa102-6dc7-46a5-8fd7-daec08fb4400,2965,fef76bd5-8447-4f20-aab9-453c1e284753
